In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
from tensorflow import keras
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'tf: {tf.__version__}, keras: {keras.__version__}')

In [ ]:
# for managing all model metadata, use neptune.ai:
import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback
neptune.__version__

In [ ]:
import numpy as np

import sys
sys.path.insert(0, '..')
import ml_utils
import pose_utils
from pose_utils import DEG_TO_RAD
from pose_utils import RAD_TO_SCALED
from pose_utils import MAX_DEPTH
from pose_utils import METERS_TO_SCALED
from pose_utils import INTENSITY_TO_SCALED

In [ ]:
# tf_data_path = '/data/all_around_zones_3500_tf_data'
# tf_data_path = '/data/all_around_scout_4501_tf_data'
# tf_data_path = '/data/face_to_face_zones_2500_tf_data'
# tf_data_path = '/data/hopper_4500_tf_data'
tf_data_path = '/data/t_formation_zones_3500_tf_data'

In [ ]:
ds_train = ml_utils.load_dataset(tf_data_path + '_train', compression='GZIP')
ds_val = ml_utils.load_dataset(tf_data_path + '_val', compression='GZIP')
ds_test = ml_utils.load_dataset(tf_data_path + '_test', compression='GZIP')

n_channels = 4

In [ ]:
### (optional) 
### test performance with a subset of input data 
### (instead of RGBD, try other combinations and color spaces)
# import cv2
# def remove_band(image, b):
#     image = np.delete(image, b, -1)
#     return image

# def rgb_to_hsv(image):
#     image = image.numpy() * 255.
#     hsv = cv2.cvtColor(image.astype('uint8'), cv2.COLOR_RGB2HSV) / 255.
#     return hsv

# def rgbd_to_hsvd(image):
#     image = image.numpy()
#     img = image[:,:,:3] * 255.
#     hsv = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2HSV) / 255.
#     image[:,:,:3] = hsv
#     return image

# def rgbd_to_hvd(image):
#     img = image[:,:,:3].numpy() * 255.
#     hsv = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2HSV) / 255.
#     hv = remove_band(hsv, 1)
#     image = remove_band(image, 1)
#     image[:,:,:2] = hv
#     return image

# def rgbd_to_hv(image):
#     image = image.numpy()
#     img = image[:,:,:3] * 255.
#     hsv = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2HSV) / 255.
#     image[:,:,:3] = hsv
#     return remove_band(image, 1)[:,:,:2]

# def rgbd_to_hd(image):
#     image = image.numpy()
#     img = image[:,:,:3] * 255.
#     hsv = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2HSV) / 255.
#     image[:,:,:3] = hsv
#     image = remove_band(image, 1) # remove s
#     image = remove_band(image, 1) # remove v
#     return image

# n_channels = 1
# mapper = lambda image, label: (image[:,:,-1], label)  # only Depth
# n_channels = 3
# mapper = lambda image, label: (image[:,:,:3], label)  # only RGB
# n_channels = 3
# mapper = lambda image, label: (tf.py_function(func=remove_band, inp=[image, 1], Tout=tf.float64), label)  # only Red + Blue + Depth
# n_channels = 2
# mapper = lambda image, label: (tf.py_function(func=remove_band, inp=[image[:,:,:3], 1], Tout=tf.float64), label)  # only Red + Blue
# n_channels = 3
# mapper = lambda image, label: (tf.py_function(func=rgb_to_hsv, inp=[image[:,:,:3]], Tout=tf.float64), label)  # HSV
# n_channels = 4
# mapper = lambda image, label: (tf.py_function(func=rgbd_to_hsvd, inp=[image], Tout=tf.float64), label)  # HSV + Depth
# n_channels = 3
# mapper = lambda image, label: (tf.py_function(func=rgbd_to_hvd, inp=[image], Tout=tf.float64), label)  # only Hue and Value + Depth
# n_channels = 2
# mapper = lambda image, label: (tf.py_function(func=rgbd_to_hv, inp=[image], Tout=tf.float64), label)  # only Hue and Value
# n_channels = 2
# mapper = lambda image, label: (tf.py_function(func=rgbd_to_hd, inp=[image], Tout=tf.float64), label)  # only Hue + Depth
# ds_train = ds_train.map(mapper)
# ds_val = ds_val.map(mapper)
# ds_test = ds_test.map(mapper)

In [ ]:
for image, label in ds_train.take(1):
    pose_utils.show_rgbd(image, format='rgbd')
    d, theta, yaw = label.numpy()
    print(f"raw:   d = {d:.3f} , theta = {theta:.3f}    , yaw = {yaw:.3f} ")
    print(f"human: d = {d / METERS_TO_SCALED:.2f} m, theta = {theta / RAD_TO_SCALED / DEG_TO_RAD:.1f} deg, yaw = {yaw / RAD_TO_SCALED / DEG_TO_RAD:.1f} deg")

In [ ]:
PROJECT = 'ljburtz/relative-pose'
PARAMS = {
    'height': 120,
    'width': 160,
    'channels': n_channels,
    'pool_size' : 2,
    'patience': 4,
    'batch_size': 32,
    'epochs': 300,
    'alpha': 0.1,
    'beta': 0.03,
    'description': 'no preprocessing pipeline'
}
# TAGS = ['all_around']
# TAGS = ['all_around_scout']
# TAGS = ['face_to_face']
# TAGS = ['hopper']
TAGS = ['t_formation']

## Build the model

In [ ]:
from tensorflow.keras import layers

In [ ]:
def make_basic_model(input_shape, height, width, n_outputs, pool_size):

    model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.experimental.preprocessing.Resizing(height, width),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=pool_size),        
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Flatten(),

        layers.Dense(6, activation="relu"),
        layers.Dense(24, activation="relu"),
        layers.Dense(24, activation="relu"),
        layers.Dropout(0.1),
        layers.Dense(n_outputs, activation="linear"),
    ]
)
    return model

In [ ]:
# input_shape = (28, 28, 1) # for MNIST
# input_shape = (32, 32, 3)  # for CIFAR
input_shape = (480, 640, n_channels)
height, width = PARAMS['height'], PARAMS['width']
pool_size = PARAMS['pool_size']
n_outputs = 3

model = make_basic_model(input_shape, height, width, n_outputs, pool_size)
model.summary()
keras.utils.plot_model(model, show_shapes=True)

## Train the model

In [ ]:
batch_size = PARAMS['batch_size']
buffer_size = tf.data.experimental.AUTOTUNE  # the prefetch buffer size is dynamically tuned

ds_train_b = ds_train.batch(batch_size, drop_remainder=True).prefetch(buffer_size)
ds_val_b = ds_val.batch(batch_size, drop_remainder=True).prefetch(buffer_size)
ds_test_b = ds_test.batch(batch_size, drop_remainder=True).prefetch(buffer_size)

In [ ]:
epochs = PARAMS['epochs']
alpha = PARAMS['alpha']
beta = PARAMS['beta']

def pose_loss(y_true, y_pred):
    pose_loss = \
        ml_utils.distance_loss(y_true, y_pred) +  \
        alpha * ml_utils.theta_loss(y_true, y_pred) + \
        beta * ml_utils.orientation_loss(y_true, y_pred)
    return pose_loss

def theta_loss(y_true, y_pred):
    return alpha * ml_utils.theta_loss(y_true, y_pred)

def orientation_loss(y_true, y_pred):
    return beta * ml_utils.orientation_loss(y_true, y_pred)

model.compile(
    loss=pose_loss, 
    optimizer='adam', 
    metrics=[
        ml_utils.distance_loss,     # intermediate loss, for tuning alpha and beta
        theta_loss,                 # intermediate loss, for tuning alpha and beta
        orientation_loss,           # intermediate loss, for tuning alpha and beta
        ml_utils.distance_diff,     # intermediate errors: human understandable
        ml_utils.theta_diff,        # intermediate errors: human understandable
        ml_utils.orientation_diff,  # intermediate errors: human understandable
    ]
)

In [ ]:
PARAMS, TAGS

In [ ]:
# create a run to log all data to the neptune cloud
run = neptune.init(project=PROJECT,
                   tags=TAGS,
                   source_files=['RelativePose_train.ipynb']  # upload a snapshot of the notebook
#                    mode='debug'
                  )
run_id = run['sys/id'].fetch()
print(run_id)
run.assign({'parameters': PARAMS}, wait=True)  # synchronous call to make sure parameters are synced with the neptune server

In [ ]:
callbacks = [
    # callback for model metadata logging
    NeptuneCallback(run=run, base_namespace='metrics'),
    keras.callbacks.ModelCheckpoint(
        '/tmp/best_model_loss.h5', 
        save_weights_only=True,
        save_best_only=True, 
        monitor='val_loss', 
        mode='min'
    ),
    # usual callbacks
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
#         min_delta=0.0003,
        mode='min', 
        patience=PARAMS['patience'], 
        verbose=1, 
        restore_best_weights=True
    ),
    # callback for profiling CPU/GPU utilisation
    keras.callbacks.TensorBoard(
        log_dir='/tmp/profile', 
        profile_batch='5,15',
        histogram_freq=0, 
        write_images=False
    ),
]

In [ ]:
history = model.fit(
    ds_train_b, 
    epochs=epochs, 
    validation_data=ds_val_b,
    callbacks=callbacks,
)

In [ ]:
import json
path = f"/tmp/model_"

# save full model (e.g for resuming training later on)
model.save(path + 'end.h5')

# save model weights and architecture (e.g for inference only)
model.save_weights(path + 'relative_pose_weights.h5')
json_config = model.to_json()
with open(path + 'relative_pose_config.json', 'w') as out_:
    json.dump(json_config, out_)

# log artifacts to Neptune    
run["model/model_end"].upload(path + 'end.h5')
run['model/model_relative_pose_weights'].upload(path + "relative_pose_weights.h5")
run['model/model_relative_pose_config'].upload(path + "relative_pose_config.json")

# log additional metrics
min_loss, min_loss_epoch, min_pdiff, min_pdiff_epoch = ml_utils.get_best_metrics(history, accuracy_metric='distance_diff')
run['metrics/min_loss'] = min_loss
run['metrics/min_loss_epoch'] = min_loss_epoch
run["metrics/min_pdiff"] = min_pdiff
run['metrics/min_pdiff_epoch'] = min_pdiff_epoch
print(run['sys/id'].fetch())

evaluate model on test dataset

In [ ]:
results = model.evaluate(ds_test_b)
print("test loss, test metrics:", results)
run['test/avg_position_diff'] = results[-3]
run['test/avg_theta_diff'] = results[-2]
run['test/avg_orientation_diff'] = results[-1]

In [ ]:
n_pred = 100
d_true, theta_true, yaw_true, d_list, theta_list, yaw_list = ml_utils.predict_and_scale(
    model,
    ds_test,
    ds_test_b,
    n_pred,
    batch_size
)

In [ ]:
fig = pose_utils.compare_each_output( 
    d_true, theta_true, yaw_true, 
    d_list, theta_list, yaw_list,
    subset=50
)
fig

In [ ]:
run['test/compare_outputs'].upload(neptune.types.File.as_html(fig))  # interactive fig
run['test/compare_outputs_png'].upload(neptune.types.File.as_image(pose_utils.plotly2array(fig)))  # static fig

In [ ]:
fig = pose_utils.compare_optical_poses(
    d_true, theta_true, yaw_true, 
    d_list, theta_list, yaw_list, 
    yaw_viz_offset=np.pi/2.,
#     footprint='small_scout_1'
#     footprint='processing_plant'
    footprint='small_excavator_1'
)
fig

In [ ]:
run['test/compare_poses'].upload(neptune.types.File.as_html(fig))  # interactive fig
run['test/compare_poses_png'].upload(neptune.types.File.as_image(pose_utils.plotly2array(fig)))  # static fig

In [ ]:
fig = pose_utils.hist_errors(
    d_true, theta_true, yaw_true, 
    d_list, theta_list, yaw_list
)

In [ ]:
run['test/error_hist'].upload(neptune.types.File.as_image(fig))

In [ ]:
run.stop()